In [24]:

import requests
from datetime import datetime, timedelta
import uuid
import sqlite3 
import hashlib

headers = {
    'X-Auth-Token': 'c0efea6535bc472fbc39c9f1ca96fa91'
}
url = 'http://api.football-data.org/v2/teams/86/matches?status=FINISHED'

response = requests.get(url, headers=headers)
matches = response.json()

for match in matches['matches']:
    start_time = datetime.strptime(match['utcDate'], '%Y-%m-%dT%H:%M:%SZ')
    # Assuming 90 minutes + 15 minutes for half-time break
    finish_time = start_time + timedelta(minutes=105)
    result = f"{match['homeTeam']['name']} vs {match['awayTeam']['name']} - {match['score']['fullTime']['homeTeam']}:{match['score']['fullTime']['awayTeam']}"
    winner = match['score']['winner']
    home_team = match['homeTeam']['name']
    awayTeam = match['awayTeam']['name']
    if winner == 'HOME_TEAM':
        winner = match['homeTeam']['name']
    elif winner == 'AWAY_TEAM':
        winner = match['awayTeam']['name']
    else:
        winner = "Draw"
    
    to_b_hashed = f"{home_team}{away_team}".encode('utf-8')
    new_uuid = hashlib.sha256(to_b_hashed).hexdigest()     
    conn = sqlite3.connect('../db/players.db')

    cursor = conn.cursor()
    insert_query = '''
    INSERT INTO event_results (ts, winner, end_date, uuid) 
    VALUES (?, ?, ?, ?);
    '''
    cursor.execute(insert_query, (start_time, winner, finish_time, str(new_uuid)))


    conn.commit()
    conn.close()

    print(f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}, Finish Time: {finish_time.strftime('%Y-%m-%d %H:%M:%S')}, Result: {result}, Winner: {winner}")


C:\Users\noelj\AppData\Local\Temp\ipykernel_8116\851769305.py:39: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(insert_query, (start_time, winner, finish_time, str(new_uuid)))


Start Time: 2023-08-12 19:30:00, Finish Time: 2023-08-12 21:15:00, Result: Athletic Club vs Real Madrid CF - 0:2, Winner: Real Madrid CF
Start Time: 2023-08-19 17:30:00, Finish Time: 2023-08-19 19:15:00, Result: UD Almería vs Real Madrid CF - 1:3, Winner: Real Madrid CF
Start Time: 2023-08-25 19:30:00, Finish Time: 2023-08-25 21:15:00, Result: RC Celta de Vigo vs Real Madrid CF - 0:1, Winner: Real Madrid CF
Start Time: 2023-09-02 14:15:00, Finish Time: 2023-09-02 16:00:00, Result: Real Madrid CF vs Getafe CF - 2:1, Winner: Real Madrid CF
Start Time: 2023-09-17 19:00:00, Finish Time: 2023-09-17 20:45:00, Result: Real Madrid CF vs Real Sociedad de Fútbol - 2:1, Winner: Real Madrid CF
Start Time: 2023-09-20 16:45:00, Finish Time: 2023-09-20 18:30:00, Result: Real Madrid CF vs 1. FC Union Berlin - 1:0, Winner: Real Madrid CF
Start Time: 2023-09-24 19:00:00, Finish Time: 2023-09-24 20:45:00, Result: Club Atlético de Madrid vs Real Madrid CF - 3:1, Winner: Club Atlético de Madrid
Start Time:

In [20]:
#Get upcoming
import sqlite3 
import uuid
import hashlib


url = 'http://api.football-data.org/v2/teams/86/matches?status=SCHEDULED'

response = requests.get(url, headers=headers)
matches = response.json()


for match in matches['matches']:
    home_team = match['homeTeam']['name']
    away_team = match['awayTeam']['name']
    home_score = match['score']['fullTime']['homeTeam']
    away_score = match['score']['fullTime']['awayTeam']
    start_time = datetime.strptime(match['utcDate'], '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')
    # End time not provided directly by API, example assumes 90 minutes + start time for simplicity
    finished_normal_time = 'Yes' if match['status'] == 'FINISHED' else 'No'
    result = f"{home_team} vs {away_team} - {home_score}:{away_score}"
    

    to_b_hashed = f"{home_team}{away_team}".encode('utf-8')
    new_uuid = hashlib.sha256(to_b_hashed).hexdigest()   #Does not have nonce or a good way to hash as this is an mvp and we have many other blockers but will try to add cooler logic as is a crypto hackathon
    conn = sqlite3.connect('../db/players.db')

    cursor = conn.cursor()

    insert_query = '''
    INSERT INTO next_events (ts, team1, team2, end_date, uuid) 
    VALUES (?, ?, ?, ?, ?);
    '''
    cursor.execute(insert_query, (start_time, home_team, away_team, finished_normal_time, str(new_uuid)))


    conn.commit()
    conn.close()
 
    print(f"Start Time: {start_time}, Result: {result}")
    

Start Time: 2024-03-31 19:00:00, Result: Real Madrid CF vs Athletic Club - None:None
Start Time: 2024-04-09 19:00:00, Result: Real Madrid CF vs Manchester City FC - None:None
Start Time: 2024-04-14 00:00:00, Result: RCD Mallorca vs Real Madrid CF - None:None
Start Time: 2024-04-17 19:00:00, Result: Manchester City FC vs Real Madrid CF - None:None
Start Time: 2024-04-21 00:00:00, Result: Real Madrid CF vs FC Barcelona - None:None
Start Time: 2024-04-28 00:00:00, Result: Real Sociedad de Fútbol vs Real Madrid CF - None:None
Start Time: 2024-05-05 00:00:00, Result: Real Madrid CF vs Cádiz CF - None:None
Start Time: 2024-05-12 00:00:00, Result: Granada CF vs Real Madrid CF - None:None
Start Time: 2024-05-15 00:00:00, Result: Real Madrid CF vs Deportivo Alavés - None:None
Start Time: 2024-05-19 00:00:00, Result: Villarreal CF vs Real Madrid CF - None:None
Start Time: 2024-05-26 00:00:00, Result: Real Madrid CF vs Real Betis Balompié - None:None
